In [21]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
import openai
import pandas
pandas.__version__

import os
os.environ
envir=os.environ

api_key = os.environ.get("OPENAI_API_KEY")
#print(api_key)

In [22]:
#import os

# Set the API key
#os.environ["OPENAI_API_KEY"] = 

# Verify the key is set correctly
#print(os.getenv("OPENAI_API_KEY"))


In [23]:
from openai import OpenAI

In [24]:
client = OpenAI()

In [25]:


response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It's possible to enroll in a course even after it has started, depending on the institution's policies and the course's specific guidelines. Here are some steps you can take:\n\n1. **Check Enrollment Policies:** Review the course details on the institution's website to see if there is a specified deadline for late enrollment.\n  \n2. **Contact the Instructor:** Reach out to the course instructor or professor to explain your situation and inquire if they would allow a late enrollment. \n\n3. **Speak with the Registrar:** The registrar's office can provide information on the official policies regarding late registration and any potential fees that may be involved.\n\n4. **Catch Up on Material:** If you're allowed to enroll late, be prepared to catch up on any missed content. Ask the instructor for guidance on what you need to focus on first to get up to speed.\n\n5. **Access Resources:** Utilize any available course materials, recordings, or online resources to help you catch up.\n\nWou

In [26]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [27]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [28]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [29]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
rag(query)

'To run Kafka producer, consumer, or kstreams in the terminal for a Java project, navigate to the project directory and execute the following command:\n\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your JAR file. This command will run the specified Java class (e.g., `JsonProducer.java`) with the required classpath including your JAR file and the output directory.'

In [31]:
rag('the course has already started, can I still enroll?')

"Yes, even if you don't register at the start, you're still eligible to submit the homework. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [32]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [33]:
from elasticsearch import Elasticsearch

In [34]:
es_client = Elasticsearch('http://localhost:9200') 

In [36]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"


es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [37]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [38]:
from tqdm.auto import tqdm

In [39]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [40]:
query = 'I just disovered the course. Can I still join it?'

In [41]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [42]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [43]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [44]:
rag(query)

'Yes, you can still join the course even if it has already started. However, note that you may not be able to submit some of the homework. To be eligible for a certificate, you will need to submit 2 out of 3 course projects and review 3 peers’ projects by the deadline.'

In [45]:
rag("I am interested in MLOP zoomcamp ")
print(_)

Yes, you can still join the course even if it has already started. However, note that you may not be able to submit some of the homework. To be eligible for a certificate, you will need to submit 2 out of 3 course projects and review 3 peers’ projects by the deadline.


In [46]:
def rag_general(query):
    #search_results = elastic_search(query)
    #prompt = build_prompt(query, search_results)

    
    answer = llm(query)
    return answer

In [47]:
q="who killed sir david amess's?"

In [48]:
answer=rag_general(q)
print(answer)

Sir David Amess, a British Member of Parliament, was tragically killed on October 15, 2021. The individual responsible for his death is Ali Harbi Ali, who was found guilty of his murder. Ali, a British citizen of Somali heritage, was motivated by extremist Islamist views. The attack occurred during one of Sir David's constituency surgeries in Leigh-on-Sea, Essex. The case has been widely covered in the media and has had significant implications for the security measures surrounding public officials in the UK.


In [51]:
print(_)

Yes, you can still join the course even if it has already started. However, note that you may not be able to submit some of the homework. To be eligible for a certificate, you will need to submit 2 out of 3 course projects and review 3 peers’ projects by the deadline.


In [50]:
a =(1,2)
print(a)

(1, 2)
